In [1]:
import pandas as pd
import numpy as np

# Calculate changes months

In [11]:
# Read the CSV file
principal_rosters = pd.read_csv("../data-processed/updates/feb_principal_rosters.csv")


## Backfill NAs
First we're going to take our backfilled data from R and we'll deal with sandwiched NA values. If an NA values (or multiple NA values are between two matching values, we'll fill in those gaps in the rosters data)

In [13]:

def replace_na_with_nearest_match(data):
    # Convert data to pandas DataFrame
    df = pd.DataFrame(data)
    
    # Iterate over each row
    for idx, row in df.iterrows():
        # Iterate over each column
        for col_idx, col in enumerate(row):
            # If the value is NA
            if pd.isna(col):
                # Find the nearest non-NA value to the left
                left_idx = col_idx - 1
                while left_idx >= 0 and pd.isna(row[left_idx]):
                    left_idx -= 1
                left_value = row[left_idx] if left_idx >= 0 else None
                
                # Find the nearest non-NA value to the right
                right_idx = col_idx + 1
                while right_idx < len(row) and pd.isna(row[right_idx]):
                    right_idx += 1
                right_value = row[right_idx] if right_idx < len(row) else None
                
                # If both left and right values match, replace NA with the match
                if left_value == right_value:
                    df.at[idx, df.columns[col_idx]] = left_value
                    
    return df


In [14]:
principal_rosters_na_backfilled = replace_na_with_nearest_match(principal_rosters)

principal_rosters_na_backfilled.head(10)

,campus,campus_name,nes_flag,division,principal_june,principal_july,principal_aug,principal_sept,principal_oct,principal_nov,principal_dec,principal_jan,principal_feb
0,1,austin hs,NES Fall '24,central,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna
1,2,bellaire hs,Not NES,west,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli
2,3,northside hs,NES Fall '24,central,cecilia gonzales,cecilia gonzales,victor garcia,victor garcia,victor garcia,victor garcia,victor garcia,victor garcia,victor garcia
3,4,furr hs,NES-Aligned,central,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran
4,6,jones hs,Not NES,south,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco
5,7,kashmere hs,NES,north,brandon dickerson,brandon dickerson,brandon dickerson,brandon dickerson,brandon dickerson,brandon dickerson,brandon dickerson,brandon dickerson,brandon dickerson
6,8,lamar hs,Not NES,central,rita graves,rita graves,rita graves,rita graves,rita graves,rita graves,rita graves,rita graves,rita graves
7,9,wisdom hs,NES-Aligned,west,kenneth brantley,kenneth brantley,kenneth brantley,kenneth brantley,NaN,reginald bush,reginald bush,reginald bush,reginald bush
8,10,madison hs,NES-Aligned,south,yolanda bruce,yolanda bruce,tanisha lee,tanisha lee,tanisha lee,tanisha lee,edgar contreras,edgar contreras,edgar contreras
9,11,milby hs,Not NES,south,ruth pena,ruth pena,ruth pena,ruth pena,ruth pena,ruth pena,ruth pena,ruth pena,ruth pena


In [15]:
principal_rosters_na_backfilled[principal_rosters_na_backfilled['campus_name'] == 'burrus es']

,campus,campus_name,nes_flag,division,principal_june,principal_july,principal_aug,principal_sept,principal_oct,principal_nov,principal_dec,principal_jan,principal_feb
86,125,burrus es,NES-Aligned,north,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams


## Calculate changes
Now that we've backfilled our data, we'll use Python to check for months with changes and save the months where they occurred for us.

In [16]:
# Read the CSV file
df = principal_rosters_na_backfilled

# Initialize an empty list to store changes columns for each row
incoming_months_list = []
outgoing_months_list = []
incoming_principals_list = []
principal_cols = [col for col in df if col.startswith('principal_')]

#changes = pd.DataFrame()
changes_rows = [] # initialize empty list to store changes rows for our changes df

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    prev_name = None
    incoming_months = []  # Initialize changes columns for the current row
    outgoing_month_columns = []  # Initialize outgoing month columns for the current row
    principals_columns = []  # Initialize principal changes columns for the current row
    
    # Iterate over each name column
    for col in principal_cols:
        current_name = row[col]
        
        # Skip NA values
        if pd.isna(current_name) and col != 'principal_feb':
            continue
        
        # Check if there is a change from the previous name
        if (prev_name is not None and current_name != prev_name):
            incoming_months.append(col.replace('principal_', ''))
            ## find closest non na value to the left
            left_idx = principal_cols.index(col) - 1
            while left_idx >= 0 and pd.isna(row[principal_cols[left_idx]]):
                left_idx -= 1
            left_value = row[principal_cols[left_idx]] if left_idx >= 0 else None
            # append the outgoing month column to the outgoing_month_columns list
            outgoing_month_columns.append(principal_cols[left_idx].replace('principal_', ''))

            
            # Append the incoming_months list to the incoming_months_list
            change_row = {
            'campus': row['campus'], 
            'campus_name': row['campus_name'],
            # get outgoing principal and month
            'outgoing_principal': left_value,
            'outgoing_month': principal_cols[left_idx].replace('principal_', ''),
             # get incoming principal and month
            'incoming_principal': current_name,
            'incoming_month': col.replace('principal_', ''),
            }
            changes_rows.append(change_row)
        
        prev_name = current_name

    
    # Append the incoming_months list to the incoming_months_list
    incoming_months_list.append(incoming_months)
    outgoing_months_list.append(outgoing_month_columns)

    changes_only = pd.DataFrame(changes_rows)

## THIS IS WHERE WE WERE
changes_only.head(100)


,campus,campus_name,outgoing_principal,outgoing_month,incoming_principal,incoming_month
0,3,northside hs,cecilia gonzales,july,victor garcia,aug
1,9,wisdom hs,kenneth brantley,sept,reginald bush,nov
2,10,madison hs,yolanda bruce,july,tanisha lee,aug
3,10,madison hs,tanisha lee,nov,edgar contreras,dec
4,19,worthing hs,everett hare,july,alexandria gregoire,sept
...,...,...,...,...,...,...
68,382,reagan ed ctr pk-8,lauren mailhiot,june,gloria robayo,aug
69,456,high school ahead acad ms,john flowers,aug,megan lyndersay,sept
70,479,shadydale es,teri hampton,july,pandora agnew,aug
71,485,middle college hs - fraga,federico hernandez,oct,NaN,feb


Spread changes to new columns

In [17]:
# Add the changes_columns_list as a new column to the dataframe
df['incoming_months_columns'] = incoming_months_list

# Unnest the values in the `changes_columns` and spread them to new columns
max_changes = df['incoming_months_columns'].apply(len).max()
print(max_changes)
for i in range(1, max_changes + 1):
    df[f'incoming_month_{i}'] = df['incoming_months_columns'].apply(lambda x: x[i-1] if len(x) >= i else None)

# Drop the original `changes_columns` column
#df.drop(columns=['changes_columns'], inplace=True)
    
df['outgoing_months_columns'] = outgoing_months_list
for i in range(1, max_changes + 1):
    df[f'outgoing_month_{i}'] = df['outgoing_months_columns'].apply(lambda x: x[i-1] if len(x) >= i else None)
    
df

2


,campus,campus_name,nes_flag,division,principal_june,principal_july,principal_aug,principal_sept,principal_oct,principal_nov,principal_dec,principal_jan,principal_feb,incoming_months_columns,incoming_month_1,incoming_month_2,outgoing_months_columns,outgoing_month_1,outgoing_month_2
0,1,austin hs,NES Fall '24,central,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,orlando reyna,[],None,None,[],None,None
1,2,bellaire hs,Not NES,west,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,michael niggli,[],None,None,[],None,None
2,3,northside hs,NES Fall '24,central,cecilia gonzales,cecilia gonzales,victor garcia,victor garcia,victor garcia,victor garcia,victor garcia,victor garcia,victor garcia,[aug],aug,None,[july],july,None
3,4,furr hs,NES-Aligned,central,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,tammie moran,[],None,None,[],None,None
4,6,jones hs,Not NES,south,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,janet orozco,[],None,None,[],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,480,marshall es,NES,north,lauren price,lauren price,lauren price,lauren price,lauren price,lauren price,lauren price,lauren price,lauren price,[],None,None,[],None,None
270,483,white m es,Not NES,west,lisa hernandez,lisa hernandez,lisa hernandez,lisa hernandez,lisa hernandez,lisa hernandez,lisa hernandez,lisa hernandez,lisa hernandez,[],None,None,[],None,None
271,484,middle college hs - gulfton,NaN,west,holly gibson,holly gibson,holly gibson,holly gibson,holly gibson,holly gibson,holly gibson,holly gibson,holly gibson,[],None,None,[],None,None
272,485,middle college hs - fraga,NaN,central,federico hernandez,federico hernandez,federico hernandez,federico hernandez,federico hernandez,NaN,NaN,NaN,NaN,[feb],feb,None,[oct],oct,None


### Export changes df

In [18]:
# get the last index of the principal columns
last_index = principal_cols[len(principal_cols) - 1].replace('principal_', '')

last_index

'feb'

In [22]:
df[df['campus_name'] == 'burrus es']

,campus,campus_name,nes_flag,division,principal_june,principal_july,principal_aug,principal_sept,principal_oct,principal_nov,principal_dec,principal_jan,principal_feb,incoming_months_columns,incoming_month_1,incoming_month_2,outgoing_months_columns,outgoing_month_1,outgoing_month_2
86,125,burrus es,NES-Aligned,north,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,nicole williams,[],None,None,[],None,None


In [8]:

changes_only.to_csv("../data-processed/processed-python/changes_only--" + last_index + ".csv", index=False)
df.to_csv("../data-processed/processed-python/full_principals_with_changes--" + last_index + ".csv", index=False)

In [20]:
df.count()

campus                     274
campus_name                274
nes_flag                   254
division                   274
principal_june             259
principal_july             256
principal_aug              253
principal_sept             257
principal_oct              256
principal_nov              259
principal_dec              260
principal_jan              257
principal_feb              257
incoming_months_columns    274
incoming_month_1            66
incoming_month_2             7
outgoing_months_columns    274
outgoing_month_1            66
outgoing_month_2             7
dtype: int64

In [21]:
changes_only.count()

campus                73
campus_name           73
outgoing_principal    73
outgoing_month        73
incoming_principal    68
incoming_month        73
dtype: int64